In [1]:
import time
import copy

import numpy as np

import env
import vpg

In [2]:
game_spec = dict(
    n_players=3,
    n_territories=8,
    baseline_reinforcements=3,
    n_attacks_per_turn=5,
    max_turns=100
)

In [3]:
p = vpg.VPGPlayer(0.005, 0.0005, 50)

In [4]:
players = [p, p, env.DumbPlayer()]

In [8]:
np.unique(env.play_games(game_spec, players, np.arange(1000))[0], return_counts=True)[1]

array([401, 402,  37, 160])

In [5]:
vpg.train(game_spec, players, 200, 1000, [0,1,2])

epoch 0
player 0  loss_pi: 0.216 loss_vf: 0.005 return: 0.005 ep_len: 123.740  n_games: 2000
player 1 player 2 epoch 1
player 0  loss_pi: -0.009 loss_vf: 0.002 return: 0.002 ep_len: 124.812  n_games: 2000
player 1 player 2 epoch 2
player 0  loss_pi: 0.001 loss_vf: 0.002 return: 0.003 ep_len: 124.809  n_games: 2000
player 1 player 2 epoch 3
player 0  loss_pi: 0.004 loss_vf: 0.003 return: 0.004 ep_len: 124.559  n_games: 2000
player 1 player 2 epoch 4
player 0  loss_pi: -0.005 loss_vf: 0.002 return: 0.002 ep_len: 123.814  n_games: 2000
player 1 player 2 epoch 5
player 0  loss_pi: 0.001 loss_vf: 0.002 return: 0.002 ep_len: 124.135  n_games: 2000
player 1 player 2 epoch 6
player 0  loss_pi: -0.000 loss_vf: 0.002 return: 0.002 ep_len: 124.061  n_games: 2000
player 1 player 2 epoch 7
player 0  loss_pi: 0.003 loss_vf: 0.002 return: 0.003 ep_len: 124.835  n_games: 2000
player 1 player 2 epoch 8
player 0  loss_pi: -0.003 loss_vf: 0.002 return: 0.003 ep_len: 124.703  n_games: 2000
player 1 player

In [5]:
import torch
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cuda

GeForce GTX 780 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [6]:
vpg.train(game_spec, players, 600, 1000, [0,1,2])

epoch 0
player 0  loss_pi: -0.002 loss_vf: 0.074 return: 0.349 ep_len: 95.003  n_games: 2000
player 1 player 2 epoch 1
player 0  loss_pi: -0.002 loss_vf: 0.069 return: 0.340 ep_len: 95.338  n_games: 2000
player 1 player 2 epoch 2
player 0  loss_pi: 0.010 loss_vf: 0.078 return: 0.379 ep_len: 88.739  n_games: 2000
player 1 player 2 epoch 3
player 0  loss_pi: -0.004 loss_vf: 0.071 return: 0.375 ep_len: 89.213  n_games: 2000
player 1 player 2 epoch 4
player 0  loss_pi: 0.008 loss_vf: 0.075 return: 0.387 ep_len: 86.674  n_games: 2000
player 1 player 2 epoch 5
player 0  loss_pi: -0.006 loss_vf: 0.077 return: 0.382 ep_len: 86.654  n_games: 2000
player 1 player 2 epoch 6
player 0  loss_pi: 0.003 loss_vf: 0.072 return: 0.398 ep_len: 83.273  n_games: 2000
player 1 player 2 epoch 7
player 0  loss_pi: -0.006 loss_vf: 0.076 return: 0.388 ep_len: 85.268  n_games: 2000
player 1 player 2 epoch 8
player 0  loss_pi: 0.005 loss_vf: 0.074 return: 0.391 ep_len: 83.472  n_games: 2000
player 1 player 2 epoch